In [1]:
from monitorcontrol import get_monitors

for i, monitor in enumerate(get_monitors()):
    with monitor:
        brightness = monitor.get_luminance()
        contrast = monitor.get_contrast()
        print(f"Monitor {i}:\n  - Brightness: {brightness}\n  - Contrast: {contrast}")

Monitor 0:
  - Brightness: 73
  - Contrast: 83


In [2]:
def set_monitor_settings(brightness: int, contrast: int):
    for monitor in get_monitors():
        with monitor:
            monitor.set_luminance(brightness)
            monitor.set_contrast(contrast)

set_monitor_settings(brightness=95, contrast=90)

In [3]:
import os

import paho.mqtt.client as mqtt
from dotenv import load_dotenv
import time

load_dotenv()

def on_connect(client, userdata, flags, reason_code, properties):
    client.subscribe("homeassistant/light/color_temp_k")
    client.subscribe("homeassistant/light/brightness_pct")

def on_message(client, userdata, msg):
    value = msg.payload.decode()
    print(f"Topic: {msg.topic}, Value: {value}")

client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
client.on_connect = on_connect
client.on_message = on_message
client.connect(host=os.environ["HA_MQTT_Address"])
client.username_pw_set(
    username=os.environ["HA_MQTT_Username"],
    password=os.environ["HA_MQTT_Password"]
)
client.loop_start()
time.sleep(5)      
client.loop_stop() 

<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

In [4]:
# Ask for a refresh
client.publish(topic="homeassistant/light/refresh")

In [5]:
monitor_config = {
    "brightness": {
        "max": 100,
        "min": 3,
        "offset": 0,
    },
    "contrast": {
        "max": 92,
        "min": 60,
        "offset": 0,
    }
}

def percent_to_monitor_value(min: int, max: int, percent: int):
    val = max - min
    val *= percent / 100
    val = round(val)
    return min + val

percent_to_monitor_value(monitor_config["brightness"]["min"], monitor_config["brightness"]["max"], 50)

51